In [1]:
import pandas as pd
import numpy as np

In [2]:
years = [2022, 2023]
new_russian_team = np.array([1083, 2006])
new_england_team = np.array([1359, 44, 62])
new_bundes_team = np.array([180, 181])
new_serie_team = np.array([512, 490, 495])

#Россия
all_dataRFPL = pd.DataFrame()
for year in years:
    rfpl_path = f'elo/csv/Russia/{year}/RFPL.csv'
    kubok_path = f'elo/csv/Russia/{year}/KUBOK.csv'

    data_RFPL = pd.read_csv(rfpl_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        fnl_path = f'elo/csv/Russia/2022/FNL.csv'
        data_fnl = pd.read_csv(fnl_path)
        data_fnl = data_fnl[data_fnl['home_team_id'].isin(new_russian_team) | data_fnl['away_team_id'].isin(new_russian_team)]
        unique_teams_RFPL = np.concatenate((data_RFPL['home_team_id'].unique(), new_russian_team))
    else:
        unique_teams_RFPL = data_RFPL['home_team_id'].unique()
        data_fnl = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_RFPL) & data_kubok['away_team_id'].isin(unique_teams_RFPL)]
    all_dataRFPL = pd.concat([all_dataRFPL, data_kubok, data_RFPL, data_fnl])

print(len(all_dataRFPL))

571


In [3]:
all_dataRFPL = all_dataRFPL.sort_values('timestamp')
all_dataRFPL.reset_index(drop=True, inplace=True)


In [4]:
#Англия
all_dataPL = pd.DataFrame()
for year in years:
    pl_path = f'elo/csv/England/{year}/PL.csv'
    kubok_path = f'elo/csv/England/{year}/cup.csv'
    carabao_path = f'elo/csv/England/{year}/carabao.csv'

    data_PL = pd.read_csv(pl_path)
    data_kubok = pd.read_csv(kubok_path)
    data_carabao = pd.read_csv(carabao_path)

    if year == 2022:
        championship_path = f'elo/csv/England/2022/championship.csv'
        data_championship = pd.read_csv(championship_path)
        data_championship = data_championship[data_championship['home_team_id'].isin(new_england_team) | data_championship['away_team_id'].isin(new_england_team)]
        unique_teams_PL = np.concatenate((data_PL['home_team_id'].unique(), new_england_team))
    else:
        unique_teams_PL = data_PL['home_team_id'].unique()
        data_championship = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_PL) & data_kubok['away_team_id'].isin(unique_teams_PL)]
    data_carabao = data_carabao[data_carabao['home_team_id'].isin(unique_teams_PL) & data_carabao['away_team_id'].isin(unique_teams_PL)]
    all_dataPL = pd.concat([all_dataPL, data_kubok, data_PL, data_carabao, data_championship])

all_dataPL = all_dataPL.sort_values('timestamp')
all_dataPL.reset_index(drop=True, inplace=True)
all_dataPL

,fixtureID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,874671,1659121200,37,Huddersfield,0.0,1.0,44,Burnley
1,874676,1659189600,1359,Luton,0.0,0.0,54,Birmingham
2,874682,1659380400,38,Watford,1.0,0.0,62,Sheffield Utd
3,867946,1659726000,52,Crystal Palace,0.0,2.0,42,Arsenal
4,867947,1659785400,36,Fulham,2.0,2.0,40,Liverpool
...,...,...,...,...,...,...,...,...
725,1035337,1702747800,44,Burnley,0.0,2.0,45,Everton
726,1035343,1702821600,48,West Ham,3.0,0.0,39,Wolves
727,1035336,1702821600,55,Brentford,1.0,2.0,66,Aston Villa
728,1035335,1702821600,42,Arsenal,2.0,0.0,51,Brighton


In [5]:
#Germany
all_dataBundes = pd.DataFrame()
for year in years:
    bundes_path = f'elo/csv/Germany/{year}/Bundes.csv'
    kubok_path = f'elo/csv/Germany/{year}/DFB.csv'

    data_bundes = pd.read_csv(bundes_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        bundes2_path = f'elo/csv/Germany/2022/Bundes2.csv'
        data_bundes2 = pd.read_csv(bundes2_path)
        data_bundes2 = data_bundes2[data_bundes2['home_team_id'].isin(new_bundes_team) | data_bundes2['away_team_id'].isin(new_bundes_team)]
        unique_teams_Bundes = np.concatenate((data_bundes['home_team_id'].unique(), new_bundes_team))
    else:
        unique_teams_Bundes = data_bundes['home_team_id'].unique()
        data_bundes2 = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_Bundes) & data_kubok['away_team_id'].isin(unique_teams_Bundes)]
    all_dataBundes = pd.concat([all_dataBundes, data_kubok, data_bundes, data_bundes2])

print(len(all_dataBundes))
all_dataBundes

530


,fixtureID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
39,958525,1666108800,173,RB Leipzig,4,0,175,Hamburger SV
40,958526,1666205100,170,FC Augsburg,2,5,157,Bayern Munich
42,958528,1666118700,167,1899 Hoffenheim,5,1,174,FC Schalke 04
45,958531,1666205100,182,Union Berlin,2,0,180,FC Heidenheim
47,958533,1666118700,181,SV Darmstadt 98,2,1,163,Borussia Monchengladbach
...,...,...,...,...,...,...,...,...
129,1049011,1702737000,164,FSV Mainz 05,0,1,180,FC Heidenheim
130,1049012,1702668600,163,Borussia Monchengladbach,2,2,162,Werder Bremen
131,1049013,1702737000,176,VfL BOCHUM,3,0,182,Union Berlin
132,1049014,1702737000,170,FC Augsburg,1,1,165,Borussia Dortmund


In [6]:
#Italt
all_dataItaly = pd.DataFrame()
for year in years:
    bundes_path = f'elo/csv/Italy/{year}/SeriaA.csv'
    kubok_path = f'elo/csv/Italy/{year}/KUBOK.csv'

    data_bundes = pd.read_csv(bundes_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        bundes2_path = f'elo/csv/Italy/2022/SeriaB.csv'
        data_bundes2 = pd.read_csv(bundes2_path)
        data_bundes2 = data_bundes2[data_bundes2['home_team_id'].isin(new_serie_team) | data_bundes2['away_team_id'].isin(new_serie_team)]
        unique_teams_Bundes = np.concatenate((data_bundes['home_team_id'].unique(), new_serie_team))
    else:
        unique_teams_Bundes = data_bundes['home_team_id'].unique()
        data_bundes2 = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_Bundes) & data_kubok['away_team_id'].isin(unique_teams_Bundes)]
    all_dataItaly = pd.concat([all_dataItaly, data_kubok, data_bundes, data_bundes2])

print(len(all_dataItaly))

677


In [7]:
all_data = pd.concat([all_dataPL, all_dataRFPL, all_dataBundes, all_dataItaly]).sort_values('timestamp')
all_data.reset_index(drop=True, inplace=True)

In [8]:
all_data

,fixtureID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1.0,1.0,596,Zenit Saint Petersburg
1,872868,1657969200,177,Jahn Regensburg,2.0,0.0,181,SV Darmstadt 98
2,879868,1657972800,555,CSKA Moscow,2.0,0.0,1084,Ural
3,879869,1657981800,1080,Orenburg,2.0,4.0,1079,Krylya Sovetov
4,879870,1657990800,1085,Akhmat Grozny,1.0,1.0,558,Spartak Moscow
...,...,...,...,...,...,...,...,...
2503,1049010,1702830600,168,Bayer Leverkusen,3.0,0.0,169,Eintracht Frankfurt
2504,1052404,1702832400,500,Bologna,2.0,0.0,497,AS Roma
2505,1049007,1702837800,157,Bayern Munich,3.0,0.0,172,VfB Stuttgart
2506,1052407,1702842300,487,Lazio,0.0,2.0,505,Inter


In [9]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

last_time_download = 1702323900
result_data = []
for index, row in all_data.iterrows():
    fixtureID = row['fixtureID']
    timestamp = row['timestamp']
    home_team_id = row['home_team_id']
    home_goals = row['home_goals']
    away_team_id = row['away_team_id']
    away_goals = row['away_goals']
    
    if (row['timestamp'] > last_time_download):
        for team_info, position in [((home_team_id, home_goals), 'home'), ((away_team_id, away_goals), 'away')]:
            conn.request("GET", f"/fixtures/statistics?fixture={fixtureID}&team={team_info[0]}", headers=headers)
            res = conn.getresponse()
            data = res.read()
            json_data = json.loads(data)

            if 'response' in json_data and json_data['response']:
                json_data = json_data['response']
                if json_data and len(json_data) > 0 and 'statistics' in json_data[0]:
                    stat = json_data[0]['statistics']
                else:
                    stat = []
            else:
                stat = []

            if len(stat) > 0:
                # Получение ударов в створ и XG
                shots_on_target = [item['value'] for item in stat if item['type'] == 'Shots on Goal']
                if shots_on_target and isinstance(shots_on_target[0], int):
                    shots_on_target = shots_on_target[0]
                else:
                    shots_on_target = team_info[1]

                xg = [item['value'] for item in stat if item['type'] == 'expected_goals']
                if xg and isinstance(xg[0], str):
                    xg = xg[0]
                else:
                    xg = team_info[1]

                result_data.append([fixtureID, team_info[0], position, team_info[1], shots_on_target, xg, timestamp])

result_df = pd.DataFrame(result_data, columns=['fixtureID', 'team_id', 'position', 'goals', 'shots', 'xg', 'time'])

print(result_df)

    

    fixtureID  team_id position  goals  shots    xg        time
0     1049012      163     home    2.0      4  1.47  1702668600
1     1049012      162     away    2.0      5  1.67  1702668600
2     1052406      495     home    1.0      3  0.82  1702669500
3     1052406      496     away    1.0      4  1.67  1702669500
4     1035342       65     home    0.0      1  1.35  1702670400
5     1035342       47     away    2.0      6  1.57  1702670400
6     1052408      867     home    2.0      6  0.66  1702735200
7     1052408      512     away    1.0      5  1.73  1702735200
8     1049014      170     home    1.0      3  1.82  1702737000
9     1049014      165     away    1.0      7  2.73  1702737000
10    1049013      176     home    3.0      7  2.94  1702737000
11    1049013      182     away    0.0      2  0.67  1702737000
12    1049011      164     home    0.0      4  1.63  1702737000
13    1049011      180     away    1.0      2  0.53  1702737000
14    1049015      181     home    0.0  

In [10]:
prev_data = pd.read_csv('match_data.csv')
prev_data

,id,fixtureID,team_id,position,goals,shots,xg,time
0,0,879867,1994,home,1.0,3.0,1.00,1657904400
1,1,879867,596,away,1.0,2.0,1.00,1657904400
2,2,872868,177,home,2.0,4.0,2.00,1657969200
3,3,872868,181,away,0.0,3.0,0.00,1657969200
4,4,879868,555,home,2.0,8.0,2.00,1657972800
...,...,...,...,...,...,...,...,...
4815,4815,1052401,502,away,1.0,8.0,1.46,1702237500
4816,4816,1052395,511,home,1.0,6.0,1.57,1702315800
4817,4817,1052395,867,away,1.0,5.0,0.78,1702315800
4818,4818,1052394,490,home,2.0,6.0,1.09,1702323900


In [11]:
result_df = pd.DataFrame(result_data, columns=['fixtureID', 'team_id', 'position', 'goals', 'shots', 'xg', 'time'])
result_df = pd.concat([prev_data, result_df], ignore_index=True)
result_df = result_df.drop('id', axis=1)
result_df

,fixtureID,team_id,position,goals,shots,xg,time
0,879867,1994,home,1.0,3.0,1.0,1657904400
1,879867,596,away,1.0,2.0,1.0,1657904400
2,872868,177,home,2.0,4.0,2.0,1657969200
3,872868,181,away,0.0,3.0,0.0,1657969200
4,879868,555,home,2.0,8.0,2.0,1657972800
...,...,...,...,...,...,...,...
4871,1049007,172,away,0.0,2.0,0.41,1702837800
4872,1052407,487,home,0.0,4.0,1.49,1702842300
4873,1052407,505,away,2.0,4.0,1.62,1702842300
4874,1052403,499,home,4.0,11.0,1.74,1702928700


In [12]:
result_df['xg'] = result_df['xg'].astype('float')
result_df.to_csv('match_data.csv',float_format='%.2f', mode='w')

In [13]:
prev_data = pd.read_csv('match_data.csv')
prev_data

,Unnamed: 0,fixtureID,team_id,position,goals,shots,xg,time
0,0,879867,1994,home,1.0,3.0,1.00,1657904400
1,1,879867,596,away,1.0,2.0,1.00,1657904400
2,2,872868,177,home,2.0,4.0,2.00,1657969200
3,3,872868,181,away,0.0,3.0,0.00,1657969200
4,4,879868,555,home,2.0,8.0,2.00,1657972800
...,...,...,...,...,...,...,...,...
4871,4871,1049007,172,away,0.0,2.0,0.41,1702837800
4872,4872,1052407,487,home,0.0,4.0,1.49,1702842300
4873,4873,1052407,505,away,2.0,4.0,1.62,1702842300
4874,4874,1052403,499,home,4.0,11.0,1.74,1702928700
